## **Import Requaired Librares**

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## **Custom Function for Creating Graph**

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## **Extract Tesla Stock Data using yfinance Python Library**

In [ ]:
# Create a ticker object for Tesla (TSLA)
tesla = yf.Ticker("TSLA")

# Extracting stock information using the history function
tesla_stock_data = tesla.history(period="max")

tesla_stock_data.reset_index(inplace=True)
# Displaying the first few rows of the data
print(tesla_stock_data.head(5))

# **Extracting the Tesla Revenue Data using Webscraping**

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

tables = soup.find_all('table')
for index,table in enumerate(tables):
  if("Tesla Quarterly Revenue" in str(table)):
    table_index = index
print(table_index)

tesla_revenue = pd.DataFrame(columns=['Date','Revenue'])
for row in tables[table_index].tbody.find_all('tr'):
  col = row.find_all('td')
  if(col != []):
    date = col[0].text
    revenue = col[1].text
    tesla_revenue = tesla_revenue.append({'Date':date, 'Revenue':revenue},ignore_index=True)
tesla_revenue


tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")
tesla_revenue


tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue


tesla_revenue.tail(5)

## **Extract GME Stock Data using yfinance Python Library**

In [ ]:
gamestop = yf.Ticker('GME')
# Extracting stock information using the history function
gme_data = gamestop.history(period = 'max')

# Reset the index of the DataFrame
gme_data.reset_index(inplace=True)

# Rename the index column to 'Date'
gme_data.rename(columns={'index': 'Date'}, inplace=True)

# Remove timezone information and keep only the date part
gme_data['Date'] = gme_data['Date'].dt.date
gme_data.head()

# **Extracting the GME Revenue Data using Webscraping**

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

# Sending a GET request to the URL
response = requests.get(url)

# Extracting the text content of the response
html_data = response.text

# Parsing the html data using BeautifulSoup
soup = BeautifulSoup(html_data, "html.parser")

# Using BeautifulSoup to extract the table with GME Revenue
tables = soup.find_all('table')
for index,table in enumerate(tables):
  if("GameStop Quarterly Revenue" in str(table)):
    table_index = index

# Using `BeautifulSoup` or the `read_html` function extract the table with `GameStop Quarterly Revenue` and store it into a dataframe named `gme_revenue`.
# The dataframe should have columns `Date` and `Revenue`.

gme_revenue = pd.DataFrame(columns=['Date','Revenue'])
for row in tables[table_index].tbody.find_all('tr'):
  col = row.find_all('td')
  if(col != []):
    date = col[0].text
    revenue = col[1].text
    gme_revenue = gme_revenue.append({'Date':date, 'Revenue':revenue},ignore_index=True)
gme_revenue

#  Make sure the comma and dollar sign is removed from the `Revenue`
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue.tail(5)

## **Plot Tesla Stock Graph**

In [ ]:
make_graph(tesla_stock_data, tesla_revenue, 'Tesla')

## **Plot GameStop Stock Graph**

In [ ]:
# Convert the Date column to datetime format in gme_revenue
gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'])
# Convert the Date column to datetime format in gme_data
gme_data['Date'] = pd.to_datetime(gme_data['Date'])


In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop')